In [ ]:
# !pip install torch numpy pandas matplotlib scikit-learn torchvision Pillow
import sys
print(sys.executable)

In [ ]:
# !pip uninstall sklearn -y

In [ ]:
# !pip install scikit-learn
# !pip show scikit-learn

# !pip install scikit-learn
# !pip install --user scikit-learn


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix

## Data Reading

In [16]:
# Define directory paths
train_dir = r"D:\College Notes\5th Sem\CS307 Machine Learning 4\Bone_Fracture_Binary_Classification\train"
val_dir = r"D:\College Notes\5th Sem\CS307 Machine Learning 4\Bone_Fracture_Binary_Classification\val"
test_dir = r"D:\College Notes\5th Sem\CS307 Machine Learning 4\Bone_Fracture_Binary_Classification\test"

In [ ]:
folder_path = 'D:/College Notes/5th Sem/CS307 Machine Learning 4/Bone_Fracture_Binary_Classification/val/fractured'

image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
image_files = image_files[:5]

fig, axes = plt.subplots(1, 5, figsize=(15, 5))

for i, image_file in enumerate(image_files):
    img_path = os.path.join(folder_path, image_file)  # Full path to the image
    img = Image.open(img_path)                       
    axes[i].imshow(img)                              
    axes[i].axis('off')                              
    axes[i].set_title(f"Image {i+1}")                

plt.tight_layout()
plt.show()

## 1. Data Preparation

In [30]:
import torchvision.transforms as transforms  # For data transformations
# Dictionary containing transformation pipelines for training and validation data
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),                           # Convert image to tensor
        transforms.CenterCrop(400),                      # Crop the center of the image to size 400x400
        transforms.RandomHorizontalFlip(),               # Randomly flip the image horizontally with a probability of 50%
        transforms.Grayscale(num_output_channels=1)]),   # Convert the image to grayscale with a single channel

    'val': transforms.Compose([
        transforms.ToTensor(),                           # Convert image to tensor
        transforms.CenterCrop(400),                      # Crop the center of the image to size 400x400
        transforms.Grayscale(num_output_channels=1)])   # Convert the image to grayscale with a single channel for validation
}

In [31]:
# Data points (training examples)
BATCH_SIZE = 64

In [ ]:
# Defining the directory path to the original dataset
original_dataset_dir = r"D:\College Notes\5th Sem\CS307 Machine Learning 4\Bone_Fracture_Binary_Classification\train"

# Listing the classes (subdirectories) in the original dataset directory
classes = os.listdir(original_dataset_dir)

# Printing the number of classes and the list of classes
print(len(classes))  
print(classes)

In [ ]:
# Create datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(root=val_dir, transform=data_transforms['val'])
test_dataset = datasets.ImageFolder(root=test_dir, transform=data_transforms['val'])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Check the size of the datasets
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")

## 2. Build the CNN Model

In [34]:
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # Grayscale input
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 50 * 50, 256)  # Adjust based on image size after pooling
        self.fc2 = nn.Linear(256, 1)  # Binary classification

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 50 * 50)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Use sigmoid for binary classification
        return x


## 3. Train the Model

In [36]:
from PIL import ImageFile

# Allow loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model, define loss function and optimizer
model = CustomCNN().to(device)
criterion = nn.BCELoss()  # For binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.float().to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.view(-1), labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.float().to(device)
                outputs = model(images)
                loss = criterion(outputs.view(-1), labels)
                val_loss += loss.item()
        
        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")

# Train the model+
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

In [ ]:
%pip install